# Decision tree from Scratch

In [167]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder     
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score,mean_absolute_error

from ucimlrepo import fetch_ucirepo 

In [6]:
# fetch adult dataset 
adult = fetch_ucirepo(id=2) 
dataset = adult.data.original
print(adult.metadata) 

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

In [37]:
def remove_nans(df):
    return df.dropna()


def convert_num_to_cat(df, n=2, ignore=None):
    '''
    Type conversion based on n number of column unique values 
    '''
    df_dropped =df.drop(columns=ignore, axis=1)
    df_converted = df_dropped.apply(lambda x: x.astype('category')\
                                            if (str(x.dtype)=='int64' )\
                                            and (len(x.unique())<=n) else x)
    for col in ignore:
        df_converted[col] = df[col]
    return df_converted


def remove_columns(df, n=10, direction='less',ignore=None):
    '''
    Remove columns applying condition on n unique values. 
    '''
    if ignore is None:
        ignore = []
    
    # Ensure ignored columns are not processed
    df_ignored = df.drop(columns=ignore, axis=1)
    
    columns_to_drop = []
    for col in df_ignored.columns:
        unique_count = len(df_ignored[col].unique())
        if direction == 'less' and unique_count <= n and str(df_ignored[col].dtype) =='int64' :
            columns_to_drop.append(col)
        elif direction == 'more' and unique_count >= n and str(df_ignored[col].dtype) =='category':
            columns_to_drop.append(col)
    
    df_dropped = df.drop(columns=columns_to_drop)
    
    # Adding the ignored columns back if they were dropped
    for col in ignore:
        if col in df.columns:
            df_dropped[col] = df[col]
    
    return df_dropped


def object_to_categorical(df):
    '''
    type conversion 'object' to categorical  
    '''
    return df.apply(lambda x : x.astype('category') if str(x.dtype)=='object' else x)

def encode_cat(df):          
    encoder = LabelEncoder()
    return df.apply(lambda x: encoder.fit_transform(x))

In [125]:
TARGET_COLUMN = 'income'
TEST_SIZE = 0.2
RANDOM_STATE = 42
N_SAMPLES = 10

In [97]:
# Preprocessing 

df = dataset

df = remove_nans(df)

#some classes has dot at the end i.e <=50K. 
df.loc[:, TARGET_COLUMN] = df[TARGET_COLUMN].apply(lambda x: x.replace('.', '') if isinstance(x, str) and '.' in x else x)

df = convert_num_to_cat(df, n=2, ignore=[TARGET_COLUMN])
df = remove_columns(df, n=10, direction='less', ignore=[TARGET_COLUMN])
df = object_to_categorical(df)
df = remove_columns(df, n=40, direction='more',ignore=[TARGET_COLUMN])
df = encode_cat(df)

In [98]:
# Train-Test split 
X = df.loc[:, df.columns != TARGET_COLUMN]
y = df[TARGET_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)
#sampling 
X_train_np, X_test_np, y_train_np, y_test_np = X_train.to_numpy()[:N_SAMPLES], X_test.to_numpy(), y_train.to_numpy()[:N_SAMPLES], y_test.to_numpy()

In [99]:
def gini_impurity(y, labels=(0, 1)):
    if len(y)==0:
        #controlling divided by zero
        return 1
    gini=0
    for k in labels:
        # true labeled mask
        indicator = y == k
        true_cases = np.sum(indicator)
        p = true_cases / len(y)
        gini += p*(1-p)
    return gini 
        

def gini_weighted_difference(y, split, labels=(0, 1)):
    #weighted average
    l_a_weight = (np.sum(split)/len(y))
    l_b_weight = (np.sum(~split)/len(y))
    
    # each split gini impurity
    gini_minus = gini_impurity(y[split])
    gini_posit = gini_impurity(y[~split])

    weighted_diff = gini_impurity(y)-(l_a_weight*gini_minus)-(l_b_weight*gini_posit)
    
    return weighted_diff

In [100]:
def grid_search(x, y):
    '''
        for each node it provides full list of possible midpoints and their impurities
    '''
    # midpoints (potential split points)
    midpoint_ls = [np.unique(x[:,idx]) for idx in range(x.shape[1])]

    # which feature each midpoints/impurities corresponds to.
    feature_array = np.hstack([np.full(len(midpt_per_feat), idx) for idx,midpt_per_feat in enumerate(midpoint_ls)])
    
    #numpying and flattening midpoints_ls  
    midpoint_array = np.hstack(midpoint_ls)
    
    #impurity reduction values corresponding to each midpoint.        
    impurity_array = np.array([gini_weighted_difference(y,x[:,feature_array[idx]] >= midpoint)
                              for idx,midpoint in enumerate(midpoint_array)],dtype='float32')
             
    return feature_array,midpoint_array,impurity_array
                                  

def find_best_split(impurities_array, midpoints_array, features_array, verbose=False):
    
    #idx of best impurity reduction score
    optimal_idx = np.argmax(impurities_array)
    feature_idx = features_array[optimal_idx]
    best_midpoint = midpoints_array[optimal_idx]

    return (feature_idx,best_midpoint)

In [132]:
class Node:
    def __init__(self, left, right, feat_idx, split_point):
        self.left = left
        self.right = right
        self.feat_idx = feat_idx #prpbablity it means the idx of feature chosen
        self.split_point = split_point
    
    def __repr__(self):
        return f'{self.__class__.__name__} x[:,{self.feat_idx}]<={self.split_point}'


class Leaf:
    def __init__(self, label):
        self.label = label
        
    def __repr__(self):
        return f'{self.__class__.__name__} {self.label}'

In [163]:
def build_tree(x, y, current_depth, max_depth=3, n_labels=2):
    if current_depth >= max_depth:
        leaf =Leaf(np.argmax(np.bincount(y)))
        return leaf
        
    feat_arr,midpoint_arr,impurt_arr = grid_search(x,y)

    feat_idx,split_point = find_best_split(impurt_arr, midpoint_arr, feat_arr)

    split_mask = x[:,feat_idx]<split_point
    left_x = x[split_mask]
    left_y = y[split_mask]
    right_x = x[~split_mask]
    right_y = y[~split_mask]

    tree = Node(build_tree(left_x, left_y, current_depth=current_depth+1) if left_y.size !=0 else None,
                build_tree(right_x, right_y, current_depth=current_depth+1)if right_y.size !=0 else None,feat_idx,split_point)
    print(tree.left)
    return tree
    
def show_tree(node):
    pass
    # TODO: implem DFS based 


def predict(node, x):
    while True:
        if node.__class__.__name__ == 'Leaf':
            return node.label
            
        feat_idx,split_point = node.feat_idx,node.split_point
        is_left = x[feat_idx] <split_point
        if is_left :
            node = node.left
            continue
        node = node.right
        
def predict_all(tree,X):
    apply_pred = lambda x_dpoint : predict(tree, x_dpoint)
    return np.apply_along_axis(apply_pred, axis=1, arr=X)

## Comparing with Scikit learn implementation 

In [155]:
# Build vanilla decision tree
tree = build_tree(X_train_np,y_train_np,0)

Leaf 0
Leaf 1
Node x[:,10]<=81
Leaf 0
Leaf 0
Node x[:,7]<=5
Node x[:,4]<=12


In [164]:
#predict with vanilla tree
predictions_train = predict_all(tree,X_train_np)
predictions_test = predict_all(tree,X_test_np)

[0 0 0 ... 1 1 0]


In [118]:
#sklearn decision tree
sk_tree = DecisionTreeClassifier(max_depth=3).fit(X_train_np,y_train_np)
sk_predictions_train = sk_tree.predict(X_train_np)
sk_predictions_test = sk_tree.predict(X_test_np)

In [165]:
# Calculate training and test scores
vanilla_acc_score_train =accuracy_score(y_train_np, predictions_train)
vanilla_acc_score_test =accuracy_score(y_test_np, predictions_test)
vanilla_preci_score =precision_score(y_test_np, predictions_test, average='macro')
vanilla_recall_score = recall_score(y_test_np, predictions_test, average='macro')
print('------------------MY MODEL -----------------')
print('Accuracy Training: ',vanilla_acc_score_train)
print('Accuracy: ',vanilla_acc_score_test)
print('Precision: ',vanilla_preci_score)
print('Recall: ',vanilla_recall_score)

sk_acc_score_train =accuracy_score(y_train_np, predictions_train)
sk_acc_score_test =accuracy_score(y_test_np, predictions_test)
sk_preci_score =precision_score(y_test_np, predictions_test, average='macro')
sk_recall_score = recall_score(y_test_np, predictions_test, average='macro')
print('------------------MY MODEL -----------------')
print('Accuracy Training: ',sk_acc_score_train)
print('Accuracy: ',sk_acc_score_test)
print('Precision: ',sk_preci_score)
print('Recall: ',sk_recall_score)



------------------MY MODEL -----------------
Accuracy Training:  0.8380144897102058
Accuracy:  0.8372703412073491
Precision:  0.8079616592624852
Recall:  0.7132812918371773
------------------MY MODEL -----------------
Accuracy Training:  0.8380144897102058
Accuracy:  0.8372703412073491
Precision:  0.8079616592624852
Recall:  0.7132812918371773


In [172]:
# measuring similarity between metrics of vanilla model and sk learn using relative difference minues one
sim_acc_train = 1-(((vanilla_acc_score_train-sk_acc_score_train)**2)/sk_acc_score_train)
sim_acc_test = 1-(((vanilla_acc_score_test-sk_acc_score_test)**2)/sk_acc_score_test)
sim_preci = 1-(((vanilla_preci_score-sk_preci_score)**2)/sk_preci_score)
sim_recall = 1-(((vanilla_recall_score-sk_recall_score)**2)/sk_recall_score)

print(sim_acc_test,sim_acc_train,sim_preci,sim_recall)

1.0 1.0 1.0 1.0
